In [2]:
%load utils_prepro.py
import utils_prepro

In [4]:
#!pip install hazm
#!pip install -q clean-text[gpl]

In [5]:
!unzip drive/MyDrive/snappfood

Archive:  drive/MyDrive/snappfood.zip
   creating: snappfood/
  inflating: __MACOSX/._snappfood    
  inflating: snappfood/test.csv      
  inflating: __MACOSX/snappfood/._test.csv  
  inflating: snappfood/dev.csv       
  inflating: __MACOSX/snappfood/._dev.csv  
  inflating: snappfood/train.csv     
  inflating: __MACOSX/snappfood/._train.csv  


In [11]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [7]:
train = pd.read_csv('snappfood/train.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
test = pd.read_csv('snappfood/test.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
dev = pd.read_csv('snappfood/dev.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
train.head()

,Unnamed: 0,comment,label,label_id
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [8]:
train = train[['comment', 'label_id']]
test = test[['comment', 'label_id']]
dev = dev[['comment', 'label_id']]
train.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


## Preprocessing text column data

In [9]:
# cleaning comments
train['cleaned_comment'] = train['comment'].apply(utils_prepro.cleaning)
test['cleaned_comment'] = test['comment'].apply(utils_prepro.cleaning)
dev['cleaned_comment'] = dev['comment'].apply(utils_prepro.cleaning)

train = train[['cleaned_comment', 'label_id']]
test = test[['cleaned_comment', 'label_id']]
dev = dev[['cleaned_comment', 'label_id']]
train.head()

,cleaned_comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


In [10]:
x_train, y_train = train['cleaned_comment'].values.tolist(), train['label_id'].values.tolist()
x_test, y_test = test['cleaned_comment'].values.tolist(), test['label_id'].values.tolist()
x_dev, y_dev = dev['cleaned_comment'].values.tolist(), dev['label_id'].values.tolist()

In [12]:
# Vectorize text reviews to numbers
vec = CountVectorizer()
X_train = vec.fit_transform(x_train)
X_test = vec.transform(x_test)

In [13]:
# Naive bayes model
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
model.score(X_test, y_test)

0.8432857142857143

In [19]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.87      0.80      0.84      3500
           1       0.82      0.89      0.85      3500

    accuracy                           0.84      7000
   macro avg       0.85      0.84      0.84      7000
weighted avg       0.85      0.84      0.84      7000



## To check some samples

In [20]:
model.predict(vec.transform(['خیلی سرد بود اصلا دوست نداشتیم']))

array([1])

In [21]:
model.predict(vec.transform(['عالی و خوشمزه بود']))

array([0])